In [1]:
import cdsapi

In [5]:
# # Example taken from:
# # https://cds-beta.climate.copernicus.eu/how-to-api#use-the-cds-api-client-for-data-access

# client = cdsapi.Client()

# dataset = 'reanalysis-era5-pressure-levels'
# request = {
#     'product_type': ['reanalysis'],
#     'variable': ['geopotential'],
#     'year': ['2024'],
#     'month': ['03'],
#     'day': ['01'],
#     'time': ['13:00'],
#     'pressure_level': ['1000'],
#     'data_format': 'grib',
# }
# target = 'download.grib'

# client.retrieve(dataset, request, target)

2024-09-06 23:15:05,807 INFO Request ID is 218af5b3-1fee-46ae-9ec8-ee046ce8bf12
2024-09-06 23:15:05,882 INFO status has been updated to accepted
2024-09-06 23:15:54,761 INFO status has been updated to successful


2e519a585f4194cb8017839b4fce8703.grib:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

'download.grib'

In [6]:

dataset = "sis-ecde-climate-indicators"
request = {
    'variable': ['mean_temperature'],
    'origin': 'reanalysis',
    'temporal_aggregation': ['yearly'],
    'spatial_aggregation': 'gridded'
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()

2024-09-14 11:45:19,618 INFO Request ID is d03495f1-159f-463b-86bd-db3814970d17
2024-09-14 11:45:19,694 INFO status has been updated to accepted
2024-09-14 11:45:21,270 INFO status has been updated to running
2024-09-14 11:45:27,023 INFO Creating download object as zip with files:
['01_mean_temperature-reanalysis-yearly-grid-1940-2023-v1.0.nc']
2024-09-14 11:45:39,825 INFO status has been updated to successful


'7a1cb6bf42e16f5188e771f1f1b744e4.zip'

In [7]:
import zipfile

In [8]:
path_to_zip_file = '/home/bogna/Projects/cassini-hackathon-2024/src/7a1cb6bf42e16f5188e771f1f1b744e4.zip'
directory_to_extract_to = '/home/bogna/Projects/cassini-hackathon-2024/src/data/7a1cb6bf42e16f5188e771f1f1b744e4'

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [2]:
#from scipy.io import netcdf
#import numpy as np
#import matplotlib
#import matplotlib.pyplot as plt
import netCDF4

In [4]:
path_to_file = '/home/bogna/Projects/cassini-hackathon-2024/src/data/7a1cb6bf42e16f5188e771f1f1b744e4/01_mean_temperature-reanalysis-yearly-grid-1940-2023-v1.0.nc'

file2read = netCDF4.Dataset(path_to_file,'r')
#file2read = netcdf.NetCDFFile(path,'r')
print(file2read.variables.keys())
print(file2read)
temp = file2read.variables['t2m']
#temp = file2read.variables['time'] # var can be 'Theta', 'S', 'V', 'U' etc..
data = temp[:]
#file2read.close()
print(data)

dict_keys(['time', 'realization', 'lat', 'lon', 't2m'])
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.7
    institution: European Centre for Medium-Range Weather Forecasts
    history: 2024-04-02T13:52 GRIB to CDM+CF via cfgrib-0.9.9.1/ecCodes-2.27.0 with {"source": "/nfs/compute-0014/data3/adaptor.mars.internal-1712065708.8353992-6424-6-39247118-b7b6-4b87-8957-46d0ba425a8f.grib", "filter_by_keys": {}, "encode_cf": ["parameter", "time", "geography", "vertical"]}
    source: ECMWF
    dimensions(sizes): time(84), lat(185), lon(271)
    variables(dimensions): int64 time(time), int64 realization(), float64 lat(lat), float64 lon(lon), float32 t2m(time, lat, lon)
    groups: 
[[[294.1182  294.092   294.07428 ... 296.2375  296.2674  296.3176 ]
  [294.06552 294.0382  294.0104  ... 296.2999  296.35367 296.4166 ]
  [294.01038 293.9791  293.94962 ... 296.2714  296.29733 296.677  ]
  ...
  [263.0305  263.28867 263.25842 ... 272.29727 2

In [5]:
time = file2read.variables['time'][:]
lat = file2read.variables['lat'][:]
lon = file2read.variables['lon'][:]
val = file2read.variables['t2m'][:]

In [6]:
def iter_3D(matrix):
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            for k in range(matrix.shape[2]):
                yield i, j, k

In [11]:
l = []

l.append('time, lat, lon, m2t')

for i, j, k in iter_3D(val):
    l.append('%d, %.2f, %.2f, %.2f' %((time[i]/365 + 1940), lat[j], lon[k], val[i, j, k]-273.15))

with open('file.csv', 'w') as f:
    f.write("\n".join(l))